In [1]:
%pip install indic-nlp-library

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer
import torch
from indicnlp.tokenize import indic_tokenize

print('All Imports Successful!')

All Imports Successful!


In [3]:
data = pd.read_csv("/kaggle/input/language-dataset/Sentence pairs in English-Hindi - 2025-02-11.tsv", sep="\t", header=None, names=["SrcSentId", "SrcSent", "DstSentId", "DstSent"])

In [4]:
data.head()

,SrcSentId,SrcSent,DstSentId,DstSent
0,1282,Muiriel is 20 now.,485968,म्यूरियल अब बीस साल की हो गई है।
1,1282,Muiriel is 20 now.,2060319,म्यूरियल अब बीस साल की है।
2,1294,Education in this world disappoints me.,485564,मैं इस दुनिया में शिक्षा पर बहुत निराश हूँ।
3,1302,That won't happen.,2060320,वैसा नहीं होगा।
4,1308,I miss you.,2060321,मुझें तुम्हारी याद आ रही है।


In [5]:
data.drop(labels=[data.columns[0], data.columns[2]], axis=1, inplace=True)

In [6]:
data.head()

,SrcSent,DstSent
0,Muiriel is 20 now.,म्यूरियल अब बीस साल की हो गई है।
1,Muiriel is 20 now.,म्यूरियल अब बीस साल की है।
2,Education in this world disappoints me.,मैं इस दुनिया में शिक्षा पर बहुत निराश हूँ।
3,That won't happen.,वैसा नहीं होगा।
4,I miss you.,मुझें तुम्हारी याद आ रही है।


In [7]:
src_sent_tokenizer = AutoTokenizer.from_pretrained("google-T5/T5-base")
print("Model Called")

config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Model Called


In [8]:
data["SrcSent"] = data["SrcSent"].apply(lambda x: src_sent_tokenizer.tokenize(x))

In [9]:
data.head()

,SrcSent,DstSent
0,"[▁Mu, i, riel, ▁is, ▁20, ▁now, .]",म्यूरियल अब बीस साल की हो गई है।
1,"[▁Mu, i, riel, ▁is, ▁20, ▁now, .]",म्यूरियल अब बीस साल की है।
2,"[▁Education, ▁in, ▁this, ▁world, ▁disappoint, ...",मैं इस दुनिया में शिक्षा पर बहुत निराश हूँ।
3,"[▁That, ▁won, ', t, ▁happen, .]",वैसा नहीं होगा।
4,"[▁I, ▁miss, ▁you, .]",मुझें तुम्हारी याद आ रही है।


In [10]:
data["DstSent"] = data["DstSent"].apply(lambda x: indic_tokenize.trivial_tokenize(x, lang="hi"))

In [11]:
data.head()

,SrcSent,DstSent
0,"[▁Mu, i, riel, ▁is, ▁20, ▁now, .]","[म्यूरियल, अब, बीस, साल, की, हो, गई, है, ।]"
1,"[▁Mu, i, riel, ▁is, ▁20, ▁now, .]","[म्यूरियल, अब, बीस, साल, की, है, ।]"
2,"[▁Education, ▁in, ▁this, ▁world, ▁disappoint, ...","[मैं, इस, दुनिया, में, शिक्षा, पर, बहुत, निराश..."
3,"[▁That, ▁won, ', t, ▁happen, .]","[वैसा, नहीं, होगा, ।]"
4,"[▁I, ▁miss, ▁you, .]","[मुझें, तुम्हारी, याद, आ, रही, है, ।]"


In [12]:
data["SrcSent"] = data["SrcSent"].apply(src_sent_tokenizer.convert_tokens_to_ids)

In [13]:
data.head()

,SrcSent,DstSent
0,"[4159, 23, 14018, 19, 460, 230, 5]","[म्यूरियल, अब, बीस, साल, की, हो, गई, है, ।]"
1,"[4159, 23, 14018, 19, 460, 230, 5]","[म्यूरियल, अब, बीस, साल, की, है, ।]"
2,"[2855, 16, 48, 296, 26963, 7, 140, 5]","[मैं, इस, दुनिया, में, शिक्षा, पर, बहुत, निराश..."
3,"[466, 751, 31, 17, 1837, 5]","[वैसा, नहीं, होगा, ।]"
4,"[27, 3041, 25, 5]","[मुझें, तुम्हारी, याद, आ, रही, है, ।]"


In [14]:
Vs = src_sent_tokenizer.get_vocab()

In [15]:
Vs

{'▁June': 1515,
 'ö': 1872,
 'KU': 18519,
 'CSA': 24135,
 '▁privé': 21611,
 'flexibel': 31339,
 '▁Ethereum': 28516,
 '▁lemn': 10538,
 '▁după': 2322,
 '▁healthier': 11960,
 '▁coating': 12258,
 'Occasionally': 29804,
 'valorile': 29235,
 '▁hot': 1312,
 '▁star': 2213,
 '▁Magnolia': 31927,
 '▁coastline': 25249,
 'abonnement': 29527,
 'P': 345,
 '▁adânc': 30779,
 '▁Personne': 22833,
 'HEN': 22735,
 '▁klicken': 27265,
 '▁published': 1790,
 '▁voyage': 6232,
 '▁opinion': 3474,
 'team': 11650,
 '▁ausgestattet': 18470,
 'WR': 15472,
 'wartet': 22906,
 'oku': 18512,
 'kunde': 20865,
 '▁preot': 25602,
 '▁Geschmack': 17735,
 '▁Shuttle': 28099,
 '▁styl': 22437,
 '▁primavara': 30366,
 '▁alle': 1854,
 'ţa': 6249,
 '▁mince': 27404,
 '▁oyster': 26716,
 'enrolled': 15097,
 '▁sold': 1916,
 'vru': 23178,
 '▁conversations': 9029,
 'Roo': 22778,
 '2006': 21196,
 '▁1967': 18148,
 '▁silicone': 21650,
 '▁condemn': 24326,
 '7': 940,
 'rien': 3483,
 'fall': 2857,
 '▁Distance': 27370,
 'b': 115,
 'CAL': 21605,
 '5

In [16]:
len(Vs)

32100

In [17]:
hindi_vocab = set()
for tokenized_hindi_sent in data["DstSent"]:
    hindi_vocab.update(tokenized_hindi_sent)

In [18]:
Vd = dict()

for idx, token in enumerate(hindi_vocab):
    Vd[token] = idx+3
Vd["<PAD>"] = 0
Vd["<SOS>"] = 1
Vd["<EOS>"] = 2

In [19]:
print(Vd)

{'घूरा': 3, 'स्वागतम्': 4, 'जुलती': 5, 'बलकानी': 6, 'कसक': 7, 'खालूँ': 8, 'स्नातक': 9, 'उदास': 10, 'डेंग्यू': 11, 'आने': 12, 'लौटेंगे': 13, 'साले': 14, 'ताज़ा': 15, 'थक': 16, 'परस्पर': 17, ':': 18, 'टहलें': 19, 'ओकोनोमियाकी': 20, 'मुताबिक': 21, 'लिपियों': 22, 'टेस्ट': 23, 'बैक': 24, 'कमर': 25, 'आँखे': 26, 'फैले': 27, 'चिड़ियाँ': 28, 'इसीलिये': 29, 'गरमी': 30, 'रिफ्रेश': 31, 'सूमो': 32, 'अफवाह': 33, 'ऋण': 34, 'नानीजी': 35, 'ताक़तवर': 36, 'खगोल': 37, 'सकी': 38, 'पढ़ते': 39, 'दीजिये': 40, 'सकतें': 41, 'रोने': 42, 'पड़ीं': 43, 'गैरेज': 44, 'किजिए': 45, 'नाचना': 46, 'अौर': 47, 'नवपाषाणकालीन': 48, 'गेंद': 49, 'फारस': 50, 'रिकॉर्ड': 51, 'द्वितीय': 52, 'द्वीप': 53, 'करवाना': 54, 'रह': 55, 'ड्राईव': 56, 'शब्दकोश': 57, 'कीव': 58, 'दीवारें': 59, 'कालोनी': 60, 'आजीवन': 61, 'कीमती': 62, 'दिखे': 63, 'सरलता': 64, 'बड़ी': 65, 'किससे': 66, 'पकड़े': 67, 'हुएँ': 68, 'सुलूक': 69, 'मारिका': 70, 'दक्षिण': 71, '१९०५': 72, 'ढकेल': 73, 'नर्सरी': 74, 'कराया': 75, 'भागता': 76, 'सुपर': 77, 'पुनर्विवाह': 78, 'जुर्

In [20]:
len(Vd)

7072

In [21]:
def convert_hindi_tokens_to_ids(hindi_sent):
    return [Vd[token] for token in hindi_sent]

In [22]:
data["DstSent"] = data["DstSent"].apply(lambda x: convert_hindi_tokens_to_ids(x))

In [23]:
data.head()

,SrcSent,DstSent
0,"[4159, 23, 14018, 19, 460, 230, 5]","[4496, 4543, 6824, 5312, 2448, 5384, 5646, 669..."
1,"[4159, 23, 14018, 19, 460, 230, 5]","[4496, 4543, 6824, 5312, 2448, 6698, 4568]"
2,"[2855, 16, 48, 296, 26963, 7, 140, 5]","[6220, 5249, 2776, 6048, 2568, 2123, 5178, 525..."
3,"[466, 751, 31, 17, 1837, 5]","[2769, 227, 4284, 4568]"
4,"[27, 3041, 25, 5]","[1519, 4444, 7065, 4701, 545, 6698, 4568]"


In [24]:
def insert_sos_token_id(hindi_sent_token_ids):
    return [1] + hindi_sent_token_ids

In [25]:
data["DstSentInput"] = data["DstSent"].apply(lambda x: insert_sos_token_id(x))

In [26]:
data.head()

,SrcSent,DstSent,DstSentInput
0,"[4159, 23, 14018, 19, 460, 230, 5]","[4496, 4543, 6824, 5312, 2448, 5384, 5646, 669...","[1, 4496, 4543, 6824, 5312, 2448, 5384, 5646, ..."
1,"[4159, 23, 14018, 19, 460, 230, 5]","[4496, 4543, 6824, 5312, 2448, 6698, 4568]","[1, 4496, 4543, 6824, 5312, 2448, 6698, 4568]"
2,"[2855, 16, 48, 296, 26963, 7, 140, 5]","[6220, 5249, 2776, 6048, 2568, 2123, 5178, 525...","[1, 6220, 5249, 2776, 6048, 2568, 2123, 5178, ..."
3,"[466, 751, 31, 17, 1837, 5]","[2769, 227, 4284, 4568]","[1, 2769, 227, 4284, 4568]"
4,"[27, 3041, 25, 5]","[1519, 4444, 7065, 4701, 545, 6698, 4568]","[1, 1519, 4444, 7065, 4701, 545, 6698, 4568]"


In [27]:
def insert_eos_token_id(hindi_sent_token_ids):
    return hindi_sent_token_ids + [2]

In [28]:
data["DstSentLabel"] = data["DstSent"].apply(lambda x: insert_eos_token_id(x))

In [29]:
data.head()

,SrcSent,DstSent,DstSentInput,DstSentLabel
0,"[4159, 23, 14018, 19, 460, 230, 5]","[4496, 4543, 6824, 5312, 2448, 5384, 5646, 669...","[1, 4496, 4543, 6824, 5312, 2448, 5384, 5646, ...","[4496, 4543, 6824, 5312, 2448, 5384, 5646, 669..."
1,"[4159, 23, 14018, 19, 460, 230, 5]","[4496, 4543, 6824, 5312, 2448, 6698, 4568]","[1, 4496, 4543, 6824, 5312, 2448, 6698, 4568]","[4496, 4543, 6824, 5312, 2448, 6698, 4568, 2]"
2,"[2855, 16, 48, 296, 26963, 7, 140, 5]","[6220, 5249, 2776, 6048, 2568, 2123, 5178, 525...","[1, 6220, 5249, 2776, 6048, 2568, 2123, 5178, ...","[6220, 5249, 2776, 6048, 2568, 2123, 5178, 525..."
3,"[466, 751, 31, 17, 1837, 5]","[2769, 227, 4284, 4568]","[1, 2769, 227, 4284, 4568]","[2769, 227, 4284, 4568, 2]"
4,"[27, 3041, 25, 5]","[1519, 4444, 7065, 4701, 545, 6698, 4568]","[1, 1519, 4444, 7065, 4701, 545, 6698, 4568]","[1519, 4444, 7065, 4701, 545, 6698, 4568, 2]"


In [30]:
data.drop(labels=[data.columns[1]], axis=1, inplace=True)

In [31]:
data.head()

,SrcSent,DstSentInput,DstSentLabel
0,"[4159, 23, 14018, 19, 460, 230, 5]","[1, 4496, 4543, 6824, 5312, 2448, 5384, 5646, ...","[4496, 4543, 6824, 5312, 2448, 5384, 5646, 669..."
1,"[4159, 23, 14018, 19, 460, 230, 5]","[1, 4496, 4543, 6824, 5312, 2448, 6698, 4568]","[4496, 4543, 6824, 5312, 2448, 6698, 4568, 2]"
2,"[2855, 16, 48, 296, 26963, 7, 140, 5]","[1, 6220, 5249, 2776, 6048, 2568, 2123, 5178, ...","[6220, 5249, 2776, 6048, 2568, 2123, 5178, 525..."
3,"[466, 751, 31, 17, 1837, 5]","[1, 2769, 227, 4284, 4568]","[2769, 227, 4284, 4568, 2]"
4,"[27, 3041, 25, 5]","[1, 1519, 4444, 7065, 4701, 545, 6698, 4568]","[1519, 4444, 7065, 4701, 545, 6698, 4568, 2]"


In [32]:
x = list(data["SrcSent"])
y_input = list(data["DstSentInput"])
y_label = list(data["DstSentLabel"])

In [33]:
x_tensor = [torch.tensor(tokenized_eng_sent_ids) for tokenized_eng_sent_ids in x]
y_input_tensor = [torch.tensor(tokenized_hindi_sent_ids) for tokenized_hindi_sent_ids in y_input]
y_label_tensor = [torch.tensor(tokenized_hindi_sent_ids) for tokenized_hindi_sent_ids in y_label]

In [34]:
x_padded = torch.nn.utils.rnn.pad_sequence(x_tensor, batch_first=True)
y_padded_input = torch.nn.utils.rnn.pad_sequence(y_input_tensor, batch_first=True)
y_padded_label = torch.nn.utils.rnn.pad_sequence(y_label_tensor, batch_first=True)

In [35]:
Ns = x_padded.shape[1]
Nd = y_padded_label.shape[1]

In [36]:
class Encoder(torch.nn.Module):
    def __init__(self, src_lang_vocab_size, word_embedding_dim):
        super(Encoder, self).__init__()
        self.first_embedding_layer = torch.nn.Embedding(num_embeddings=src_lang_vocab_size,
                                                        embedding_dim=word_embedding_dim)
        self.second_lstm_layer = torch.nn.LSTM(input_size=word_embedding_dim,
                                               hidden_size=word_embedding_dim, 
                                               batch_first=True)
    def forward(self, x_padded_mini_batch):
        first_embedding_layer_out = self.first_embedding_layer(x_padded_mini_batch)
        encoder_output, (final_encoder_output, final_cell_state) = self.second_lstm_layer(first_embedding_layer_out)

        return encoder_output, (final_encoder_output, final_cell_state)

In [37]:
class Decoder(torch.nn.Module):

    def __init__(self,dst_lang_vocab_size,word_embedding_dim):
        super(Decoder,self).__init__()

        self.first_embedding_layer = torch.nn.Embedding(num_embeddings=dst_lang_vocab_size,
                                                       embedding_dim=word_embedding_dim)
        self.second_lstm_layer = torch.nn.LSTM(input_size=word_embedding_dim,
                                               hidden_size=word_embedding_dim,
                                              batch_first=True)
        self.prediction_layer = torch.nn.Linear(in_features=word_embedding_dim,out_features=dst_lang_vocab_size)
        self.prediction_layer_activation = torch.nn.Softmax(dim=2)

    def forward(self,y_padded_input_mini_batch,final_encoder_output,final_cell_state):

        first_embedding_layer_out = self.first_embedding_layer(y_padded_input_mini_batch)
        decoder_lstm_layer_out, (final_decoder_lstm_layer_out, final_cell_state) = self.second_lstm_layer(first_embedding_layer_out,
                                                                                                         (final_encoder_output,
                                                                                                          final_cell_state))
        prediction = self.prediction_layer_activation(self.prediction_layer(decoder_lstm_layer_out))
        
        return prediction, (final_decoder_lstm_layer_out, final_cell_state)

In [38]:
class Seq2SeqEncDec(torch.nn.Module):
    def __init__(self,src_lang_vocab_size, dst_lang_vocab_size,word_embedding_dim):
        super(Seq2SeqEncDec, self).__init__()
        self.encoder = Encoder(src_lang_vocab_size,word_embedding_dim)
        self.decoder = Decoder(dst_lang_vocab_size, word_embedding_dim)
    def forward(self, x_padded_mini_batch, y_padded_input_mini_batch):
        encoder_output, (final_encoder_output, final_cell_state) = self.encoder(x_padded_mini_batch)
        y_hat_mini_batch = self.decoder(y_padded_input_mini_batch,
                                        final_encoder_output,
                                        final_cell_state)
        return y_hat_mini_batch

In [39]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(device)

cuda


In [40]:
x_padded_train = x_padded[0:13000]
y_padded_input_train = y_padded_input[0:13000]
y_padded_label_train = y_padded_label[0:13000]

x_padded_test = x_padded[13000:]
y_padded_input_test = y_padded_input[13000:]
y_padded_label_test = y_padded_label[13000:]

In [41]:
network = Seq2SeqEncDec(len(Vs),len(Vd),128).to(device)
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=0)
optimizer = torch.optim.Adam(network.parameters())
num_epochs = 25
mb_size = 65

for epoch in range(num_epochs):
    for i in range(x_padded_train.shape[0]//mb_size):

        x_train_mb = x_padded_train[i*mb_size:(i+1)*mb_size]
        y_input_mb = y_padded_input_train[i*mb_size:(i+1)*mb_size]
        y_label_mb = y_padded_label_train[i*mb_size:(i+1)*mb_size]
        y_label_mb = y_label_mb.reshape(y_label_mb.shape[0]*y_label_mb.shape[1],)
        
        x_train_mb, y_input_mb, y_label_mb = x_train_mb.to(device), y_input_mb.to(device), y_label_mb.to(device)

        y_hat_train_mb = network(x_train_mb,y_input_mb)
        y_hat_train_mb = y_hat_train_mb[0]
        y_hat_train_mb = y_hat_train_mb.reshape(y_hat_train_mb.shape[0]*y_hat_train_mb.shape[1],
                                                y_hat_train_mb.shape[2])

        loss_fn_value = loss_fn(y_hat_train_mb,y_label_mb)

        loss_fn_value.backward()
        torch.nn.utils.clip_grad_norm_(network.parameters(),max_norm=1.0)
        optimizer.step()
        optimizer.zero_grad()

        print("Epoch # {}, Time Step # {}, Loss Value = {}".format(epoch,i,loss_fn_value))

Epoch # 0, Time Step # 0, Loss Value = 8.863897323608398
Epoch # 0, Time Step # 1, Loss Value = 8.863892555236816
Epoch # 0, Time Step # 2, Loss Value = 8.863889694213867
Epoch # 0, Time Step # 3, Loss Value = 8.863885879516602
Epoch # 0, Time Step # 4, Loss Value = 8.863882064819336
Epoch # 0, Time Step # 5, Loss Value = 8.863874435424805
Epoch # 0, Time Step # 6, Loss Value = 8.863868713378906
Epoch # 0, Time Step # 7, Loss Value = 8.863861083984375
Epoch # 0, Time Step # 8, Loss Value = 8.863856315612793
Epoch # 0, Time Step # 9, Loss Value = 8.863839149475098
Epoch # 0, Time Step # 10, Loss Value = 8.8638334274292
Epoch # 0, Time Step # 11, Loss Value = 8.863807678222656
Epoch # 0, Time Step # 12, Loss Value = 8.86375904083252
Epoch # 0, Time Step # 13, Loss Value = 8.863754272460938
Epoch # 0, Time Step # 14, Loss Value = 8.863690376281738
Epoch # 0, Time Step # 15, Loss Value = 8.863643646240234
Epoch # 0, Time Step # 16, Loss Value = 8.863536834716797
Epoch # 0, Time Step # 17, 

In [42]:
torch.save(network.state_dict(), "model.pth")